In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

In [ ]:
login_url = 'https://www.jobplanet.co.kr/users/sign_in'

#email 본인 아이디, password 본인 패스워드 입력 단, 리뷰를 남겨서 전체 접근이 가능한 상태여야함
email = 'harpny49@hufs.ac.kr'
password = 'jobplanet13579'

LOGIN_INFO ={
    'user[email]' : email,
    'user[password]' : password,
    'commit' : '로그인'
}

session = requests.session()
res = session.post(login_url, data = LOGIN_INFO, verify = False)
res.raise_for_status()
result = []

/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.jobplanet.co.kr'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [ ]:
def clean_str(text):
    pattern = '([ㄱ-ㅎㅏ-ㅣ]+)'  # 한글 자음, 모음 제거
    text = re.sub(pattern=pattern, repl='', string=text)
    pattern = '<[^>]*>'         # HTML 태그 제거
    text = re.sub(pattern=pattern, repl='', string=text)
    pattern = '[^\w\s]'         # 특수기호제거
    text = re.sub(pattern=pattern, repl='', string=text)
    text = text.replace('\r','. ')
    return text

In [ ]:
rating_all = pd.read_csv('/content/drive/MyDrive/ICR_project/rating_all.csv')
rating_all['종목명'].unique()

array(['DL이앤씨', 'HDC현대산업개발', '한전KPS', '일진파워', 'HJ중공업', '신세계건설', 'KCC건설',
       '대우건설', '서희건설', '특수건설', '계룡건설산업', '신원종합개발', '서한', '태영건설', '현대건설',
       'GS건설', '동부건설', '코오롱글로벌', 'DL건설', '진흥기업', '남광토건', '한신공영', '금호건설',
       '삼부토건'], dtype=object)

In [ ]:
# url은 보고싶은 기업의 리뷰 URL이며 마지막은 ?page= 형태로 해야함
# last_page는 해당 기업 리뷰의 마지막 페이지 입력
last_page = 77
for idx in range(1,last_page+1):
    url = 'https://www.jobplanet.co.kr/companies/81985/reviews/%EB%8C%80%EC%9A%B0%EA%B1%B4%EC%84%A4?page='+str(idx)

    res = session.get(url)
    res.raise_for_status()

    soup = BeautifulSoup(res.text, 'html.parser')

    count3 = 0
    count4 = 0
    count5 = 0

    try:
        for k in range(5):
            reviewer_info = []
            # 응답자 정보
            position = soup.select('.content_top_ty2 > span.txt1')[0 + count4].text # 직무
            status = soup.select('.content_top_ty2 > span.txt1')[1 + count4].text.strip()  # 재직 여부
            loc = soup.select('.content_top_ty2 > span.txt1')[2 + count4].text      # 지역
            date = soup.select('.content_top_ty2 > span.txt1')[3+ count4].text      # 작성일

            # 점
            star_rating = soup.select('.us_star_m > div.star_score')[0+k]['style'][6:-1].replace("%", "") # 별점

            # rating 5*5
            promotion = soup.select('.bl_score')[0 + count5]['style'][6:-1].replace("%", "")  # 승진 기회 및 가능성
            welfare = soup.select('.bl_score')[1 + count5]['style'][6:-1].replace("%", "")    # 복지 및 급여
            balance = soup.select('.bl_score')[2 + count5]['style'][6:-1].replace("%", "")    # 업무와 삶의 균형
            culture = soup.select('.bl_score')[3 + count5]['style'][6:-1].replace("%", "")    # 사내문화
            top = soup.select('.bl_score')[4 + count5]['style'][6:-1].replace("%", "")        # 경영진

            # 중심 제목
            content = soup.select('h2.us_label')[0+k].text.replace("BEST", "").replace("\n","").strip()  # 총평

            # 장단점 경영진 의견
            merit = soup.select('dl.tc_list > dd.df1 > span')[0 + count3].text          # 장점
            disadvantages = soup.select('dl.tc_list > dd.df1 > span')[1 + count3].text  # 단점
            df_tit = soup.select('dl.tc_list > dd.df1 > span')[2 + count3].text         # 경영진에게 바라는 점

            review = [date, position, status, loc, star_rating, promotion, welfare, balance, culture, top,clean_str(content),clean_str(merit),clean_str(disadvantages), clean_str(df_tit)]

            result.append(review)
            review = []
            count3 += 3
            count4 += 4
            count5 += 5
            print("pass :"+str(idx)+"-"+str(k))

    except :
        print("fail :" + str(idx))
        pass

print(result)

colname = ['작성일', '직무','재직 여부','지역','총점','승진 기회 및 가능성','복지 및 급여','업무와 삶의 균형','사내문화','경영진','총평','장점','단점','바라는점']
df = pd.DataFrame(result,columns=colname)

/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.jobplanet.co.kr'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


pass :1-0
pass :1-1
pass :1-2
pass :1-3
pass :1-4


/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.jobplanet.co.kr'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


pass :2-0
pass :2-1
pass :2-2
pass :2-3
pass :2-4


/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.jobplanet.co.kr'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


...

pass :75-0
pass :75-1
pass :75-2
pass :75-3
pass :75-4


/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.jobplanet.co.kr'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


pass :76-0
pass :76-1
pass :76-2
pass :76-3
pass :76-4


/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.jobplanet.co.kr'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


pass :77-0
pass :77-1
pass :77-2
pass :77-3
pass :77-4
[['2023. 11', '인사/총무', '전직원', '서울', '60', '20', '20', '60', '80', '40', '겉만 번지르한 실속 없는 집단 비전 없이 다니기엔 더할 나위 없음', '인프라적인 복지는 최고 수준이다 사내환경 식당 어린이집 복지몰 등', '계약직이 너무 많음 실력없는 정규직 중상위층들의 태만과 실력과 자각 있는 실무자들의 빠른 이탈', '정치 말고 경영을 해주세요'], ['2023. 11', '엔지니어링', '현직원', '서울', '80', '40', '80', '60', '80', '40', '보통 건설회사보다는 인간적인 느낌이 있음', '회사 사람들이 좋다체계화된 시스템이 있다동종사 내 급여 높음', '오너가의 마인드와 역량이 의심스럽다고인물들이 너무 많다', '공정한 인사 체계 성과 및 보상 중요합니다'], ['2023. 11', '마케팅/시장조사', '현직원', '서울', '80', '60', '60', '80', '80', '60', '분업이 잘 된 대기업의 전형적인 모습', '젠틀한 사람들이 많은 편이고 회사문화도 많이 딱딱하지 않다', '중흥그룹에 먹힌 것과 그들의 경영방식에 대한 검증이 아직 이루어지지 않음', '대승적으로 생각했으면 좋겠다 더 잘 할 여지가 많은데 말이다'], ['2023. 10', '엔지니어링', '현직원', '경기', '80', '60', '80', '40', '80', '60', '급여는 타사에 비해 괜찮고 내부 분위기도 시공사치곤 괜찮은 편임', '최근 급여인상으로 괜찮아졌음 사내 조직문화와 사람들은 좋은편임', '인수 이후로 간섭이 많아지고 조직문화가 쇠퇴하는거 같음', '예전처럼 자율과 책임적 역동적인 모습으로 다시 돌아갔으면 함'], ['2023. 10', '생산관리/품질관리', '전직원', '경기', '80', '80', '80', '80', '80', '60', '건설업중에 문화는 

In [ ]:
# 저장을 희망하는 파일명으로 저장
df.to_csv("jobplanet_GS건설.csv", encoding = 'utf-8-sig', index=False)
# df.to_excel("jobplanet_대우건설.xlsx", encoding = 'utf-8', index=False)

df_result = pd.read_csv('/content/jobplanet_대우건설.xlsx')
df_result.tail()

UnicodeDecodeError: ignored